In [ ]:
import sys

# to allow imports from outside the tsp_ml/datasets/ package
sys.path.insert(0, "/home/pimenta/tsp_ml/tsp_ml")
import pandas as pd

from paths import  get_predictions_folder_path
import torch
import networkx as nx
from torch_geometric.utils.convert import to_networkx

In [ ]:
# get filepath
step="train"
instance_id = "2190b9e3f112cccbf19ece7968ec5ad2"
trained_model_name = "" # TODO
predictions_dir = get_predictions_folder_path(
    dataset_name="KEP",
    step=step,
    trained_model_name=trained_model_name,
)
predicted_instances_dir = predictions_dir / "predicted_instances"
filepath = predicted_instances_dir /(instance_id +"_pred.pt")
# load predicted instance
predicted_instance = torch.load(filepath)
# cast to nx.Graph
nx_kep_graph = to_networkx(
    data=predicted_instance,
    node_attrs=["type"],
    edge_attrs=["edge_weights", "pred", "scores"],

)
nx_kep_graph

In [ ]:
# color nodes according to their type
color_map = []
for node_id in nx_kep_graph:
    node_type = nx_kep_graph.nodes[node_id]["type"]
    if node_type == "NDD":
        color_map.append('lightgreen')
    elif node_type == 'PDP':
        color_map.append('lightblue')
    elif node_type == "P":
        color_map.append('red')
    else:
        raise ValueError(f'Type {node_type} is not valid.')
nx.draw(G=nx_kep_graph, node_color=color_map, with_labels=True)



In [ ]:
# get predicted edges
predicted_edges = []
for (edge_src, edge_dst, edge_features) in nx_kep_graph.edges(data=True):
    if edge_features['pred'] == 1:
        predicted_edges.append((edge_src, edge_dst))

In [ ]:
# plot graph
posisions_dict = nx.spring_layout(nx_kep_graph)
nx.draw(G=nx_kep_graph, pos=posisions_dict, node_color=color_map, with_labels=True)
# plot predicted route in red
nx.draw_networkx_edges(G=nx_kep_graph,pos=posisions_dict,edgelist=predicted_edges, edge_color ='r', width=3)
